# Validating Historical Features with Great Expectations

![Validating Historical Features with Great Expectations](./dqm-diagram-1.png)

In this tutorial, we will use the public dataset of Chicago taxi trips to present data validation capabilities of Feast. The original dataset is stored in BigQuery and consists of raw data for each taxi trip (one row per trip) since 2013. We will generate several training datasets (aka historical features in Feast) for different periods and evaluate expectations made on one dataset against another. Our features will represent aggregations of raw data with daily intervals (eg, trips per day, average fare or speed for a specific day, etc.). We will craft some features using SQL while pulling data from BigQuery (like total trips time or total miles travelled). Another chunk of features will be implemented using Feast's on-demand transformations - features calculated on the fly when requested.

Our plan:

0. Prepare environment
1. Pull data from BigQuery (optional)
2. Declare & apply features and feature views in Feast
3. Generate reference dataset
4. Develop & test profiler function
5. Run validation on different dataset using reference dataset & profiler

### 0. Setup

Install Feast Python SDK and great expectations:

In [ ]:
!pip install 'feast[ge]'

### 1. Dataset preparation (Optional) 

**You can skip this step if you don't have GCP account. Please use parquet files that are coming with this tutorial instead**

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
import pyarrow.parquet

from google.cloud.bigquery import Client

In [ ]:
bq_client = Client(project='kf-feast')

Running some basic aggregations while pulling data from BigQuery. Grouping by taxi_id and day:

In [ ]:
data_query = """SELECT 
    taxi_id,
    TIMESTAMP_TRUNC(trip_start_timestamp, DAY) as day,
    SUM(trip_miles) as total_miles_travelled,
    SUM(trip_seconds) as total_trip_seconds,
    SUM(fare) as total_earned,
    COUNT(*) as trip_count
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
WHERE 
    trip_miles > 0 AND trip_seconds > 60 AND
    trip_start_timestamp BETWEEN '2019-01-01' and '2020-12-31' AND
    trip_total < 1000
GROUP BY taxi_id, TIMESTAMP_TRUNC(trip_start_timestamp, DAY)"""

In [ ]:
driver_stats_table = bq_client.query(data_query).to_arrow()

# Storing resulting dataset into parquet file
pyarrow.parquet.write_table(driver_stats_table, "trips_stats.parquet")

In [ ]:
def entities_query(year):
    return f"""SELECT
    distinct taxi_id
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
WHERE
    trip_miles > 0 AND trip_seconds > 0 AND
    trip_start_timestamp BETWEEN '{year}-01-01' and '{year}-12-31'
"""

In [ ]:
entities_2019_table = bq_client.query(entities_query(2019)).to_arrow()

# Storing entities (taxi ids) into parquet file
pyarrow.parquet.write_table(entities_2019_table, "entities.parquet")

In [ ]:
#entities_2020_table = bq_client.query(entities_query(2020)).to_arrow()
#pyarrow.parquet.write_table(entities_2019_table, "entities_2020.parquet")

## 2. Declaring features

In [2]:
import pyarrow.parquet
import pandas as pd

from feast import FeatureView, Entity, FeatureStore, Field, BatchFeatureView
from feast.types import Float64, Int64
from feast.value_type import ValueType
from feast.data_format import ParquetFormat
from feast.on_demand_feature_view import on_demand_feature_view
from feast.infra.offline_stores.file_source import FileSource
from feast.infra.offline_stores.file import SavedDatasetFileStorage
from datetime import timedelta

In [3]:
batch_source = FileSource(
    timestamp_field="day",
    path="trips_stats.parquet",  # using parquet file that we created on previous step
    file_format=ParquetFormat()
)

In [4]:
taxi_entity = Entity(name='taxi', join_keys=['taxi_id'])

In [5]:
trips_stats_fv = BatchFeatureView(
    name='trip_stats',
    entities=[taxi_entity],
    schema=[
        Field(name="total_miles_travelled", dtype=Float64),
        Field(name="total_trip_seconds", dtype=Float64),
        Field(name="total_earned", dtype=Float64),
        Field(name="trip_count", dtype=Int64),

    ],
    ttl=timedelta(seconds=86400),
    source=batch_source,
)

/Users/lrangine/Documents/Src/feast/sdk/python/feast/batch_feature_view.py:72: RuntimeWarning: Batch feature views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(


*Read more about feature views in [Feast docs](https://docs.feast.dev/getting-started/concepts/feature-view)*

In [6]:
@on_demand_feature_view(
    sources=[
      trips_stats_fv,
    ],
    schema=[
        Field(name="avg_fare", dtype=Float64),
        Field(name="avg_speed", dtype=Float64),
        Field(name="avg_trip_seconds", dtype=Float64),
        Field(name="earned_per_hour", dtype=Float64),
    ]
)
def on_demand_stats(inp: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame()
    out["avg_fare"] = inp["total_earned"] / inp["trip_count"]
    out["avg_speed"] = 3600 * inp["total_miles_travelled"] / inp["total_trip_seconds"]
    out["avg_trip_seconds"] = inp["total_trip_seconds"] / inp["trip_count"]
    out["earned_per_hour"] = 3600 * inp["total_earned"] / inp["total_trip_seconds"]
    return out

*Read more about on demand feature views [here](https://docs.feast.dev/reference/alpha-on-demand-feature-view)*

In [7]:
store = FeatureStore(".")  # using feature_store.yaml that stored in the same directory

In [8]:
store.apply([taxi_entity, trips_stats_fv, on_demand_stats])  # writing to the registry

/Users/lrangine/Documents/Src/feast/sdk/python/feast/feature_store.py:534: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
/Users/lrangine/Documents/Src/feast/sdk/python/feast/batch_feature_view.py:72: RuntimeWarning: Batch feature views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(


## 3. Generating training (reference) dataset

In [9]:
taxi_ids = pyarrow.parquet.read_table("entities.parquet").to_pandas()

Generating range of timestamps with daily frequency:

In [10]:
timestamps = pd.DataFrame()
timestamps["event_timestamp"] = pd.date_range("2019-06-01", "2019-07-01", freq='D')

Cross merge (aka relation multiplication) produces entity dataframe with each taxi_id repeated for each timestamp:

In [11]:
entity_df = pd.merge(taxi_ids, timestamps, how='cross')
entity_df

,taxi_id,event_timestamp
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-01
1,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-02
2,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-03
3,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-04
4,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-05
...,...,...
156979,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2019-06-27
156980,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2019-06-28
156981,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2019-06-29
156982,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2019-06-30


Retriving historical features for resulting entity dataframe and persisting output as a saved dataset:

In [ ]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "trip_stats:total_miles_travelled",
        "trip_stats:total_trip_seconds",
        "trip_stats:total_earned",
        "trip_stats:trip_count",
        "on_demand_stats:avg_fare",
        "on_demand_stats:avg_trip_seconds",
        "on_demand_stats:avg_speed",
        "on_demand_stats:earned_per_hour",
    ]
)

store.create_saved_dataset(
    from_=job,
    name='my_training_ds',
    storage=SavedDatasetFileStorage(path='my_training_ds.parquet')
)

## 4. Developing dataset profiler

Dataset profiler is a function that accepts dataset and generates set of its characteristics. This charasteristics will be then used to evaluate (validate) next datasets.

**Important: datasets are not compared to each other! 
Feast use a reference dataset and a profiler function to generate a reference profile. 
This profile will be then used during validation of the tested dataset.**

In [13]:
import numpy as np

from feast.dqm.profilers.ge_profiler import ge_profiler

from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.dataset import PandasDataset

/usr/local/anaconda3/envs/feast-3.11/lib/python3.11/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/usr/local/anaconda3/envs/feast-3.11/lib/python3.11/site-packages/pyspark/sql/connect/utils.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(grpc.__version__) < LooseVersion(minimum_grpc_version):


Loading saved dataset first and exploring the data:

In [14]:
ds = store.get_saved_dataset('my_training_ds')
ds.to_df()

,taxi_id,event_timestamp,earned_per_hour,total_earned,trip_count,avg_fare,total_miles_travelled,avg_speed,avg_trip_seconds,total_trip_seconds
0,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-06-01 00:00:00+00:00,45.559701,203.50,8,25.437500,69.50,15.559701,2010.000000,16080
1,33164e16dd29b1c58cd15cce31df4bfcb75d9903cb66de...,2019-06-01 00:00:00+00:00,36.219512,74.25,5,14.850000,15.80,7.707317,1476.000000,7380
2,226fe0b00be42932bdff81bc0b318b883bfbf15dd48093...,2019-06-01 00:00:00+00:00,54.212598,114.75,6,19.125000,38.50,18.188976,1270.000000,7620
3,5a5bed1b5ced617d0594007d591f10bbbca354d50b19ca...,2019-06-01 00:00:00+00:00,45.000000,70.75,4,17.687500,20.22,12.860777,1415.000000,5660
4,b7f7dbb452c0fb980a0f2050a146147c1006fe5f34e3b0...,2019-06-01 00:00:00+00:00,53.783319,104.25,5,20.850000,34.49,17.793637,1395.600000,6978
...,...,...,...,...,...,...,...,...,...,...
119803,961263722c1beadafef2355412d672acac35e4054f6aaa...,2019-07-01 00:00:00+00:00,76.369295,102.25,4,25.562500,36.98,27.619917,1205.000000,4820
119804,8b07f9156e568a37d362463c84dbd1118b4eeb753bae50...,2019-07-01 00:00:00+00:00,52.677165,111.50,11,10.136364,29.00,13.700787,692.727273,7620
119805,a112879f10892d5c698ce150af17aa28615b6d005ca749...,2019-07-01 00:00:00+00:00,54.649682,143.00,16,8.937500,31.00,11.847134,588.750000,9420
119806,68fe14b9fc2d53de5ac349d47f80f43fea895e201a31e3...,2019-07-01 00:00:00+00:00,73.770492,101.25,3,33.750000,37.86,27.584699,1647.000000,4941


Feast uses [Great Expectations](https://docs.greatexpectations.io/docs/) as a validation engine and [ExpectationSuite](https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/core/expectation_suite/index.html#great_expectations.core.expectation_suite.ExpectationSuite) as a dataset's profile. Hence, we need to develop a function that will generate ExpectationSuite. This function will receive instance of [PandasDataset](https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/dataset/index.html?highlight=pandasdataset#great_expectations.dataset.PandasDataset) (wrapper around pandas.DataFrame) so we can utilize both Pandas DataFrame API and some helper functions from PandasDataset during profiling.

In [15]:
DELTA = 0.1  # controlling allowed window in fraction of the value on scale [0, 1]

@ge_profiler
def stats_profiler(ds: PandasDataset) -> ExpectationSuite:
    # simple checks on data consistency
    ds.expect_column_values_to_be_between(
        "avg_speed",
        min_value=0,
        max_value=60,
        mostly=0.99  # allow some outliers
    )

    ds.expect_column_values_to_be_between(
        "total_miles_travelled",
        min_value=0,
        max_value=500,
        mostly=0.99  # allow some outliers
    )

    # expectation of means based on observed values
    observed_mean = ds.trip_count.mean()
    ds.expect_column_mean_to_be_between("trip_count",
                                        min_value=observed_mean * (1 - DELTA),
                                        max_value=observed_mean * (1 + DELTA))

    observed_mean = ds.earned_per_hour.mean()
    ds.expect_column_mean_to_be_between("earned_per_hour",
                                        min_value=observed_mean * (1 - DELTA),
                                        max_value=observed_mean * (1 + DELTA))


    # expectation of quantiles
    qs = [0.5, 0.75, 0.9, 0.95]
    observed_quantiles = ds.avg_fare.quantile(qs)

    ds.expect_column_quantile_values_to_be_between(
        "avg_fare",
        quantile_ranges={
            "quantiles": qs,
            "value_ranges": [[None, max_value] for max_value in observed_quantiles]
        })

    return ds.get_expectation_suite()

Testing our profiler function:

In [16]:
ds.get_profile(profiler=stats_profiler)

<GEProfile with expectations: [
  {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "avg_speed",
      "min_value": 0,
      "max_value": 60,
      "mostly": 0.99
    },
    "meta": {}
  },
  {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "total_miles_travelled",
      "min_value": 0,
      "max_value": 500,
      "mostly": 0.99
    },
    "meta": {}
  },
  {
    "expectation_type": "expect_column_mean_to_be_between",
    "kwargs": {
      "column": "trip_count",
      "min_value": 10.387244591346153,
      "max_value": 12.695521167200855
    },
    "meta": {}
  },
  {
    "expectation_type": "expect_column_mean_to_be_between",
    "kwargs": {
      "column": "earned_per_hour",
      "min_value": 52.32062497564023,
      "max_value": 63.9474305257825
    },
    "meta": {}
  },
  {
    "expectation_type": "expect_column_quantile_values_to_be_between",
    "kwargs": {
      "column": "avg_fare"

**Verify that all expectations that we coded in our profiler are present here. Otherwise (if you can't find some expectations) it means that it failed to pass on the reference dataset (do it silently is default behavior of Great Expectations).**

Now we can create validation reference from dataset and profiler function:

In [17]:
validation_reference = ds.as_reference(name="validation_reference_dataset", profiler=stats_profiler)

and test it against our existing retrieval job

In [18]:
_ = job.to_df(validation_reference=validation_reference)

Validation successfully passed as no exception were raised.

### 5. Validating new historical retrieval 

Creating new timestamps for Dec 2020:

In [19]:
from feast.dqm.errors import ValidationFailed

In [20]:
timestamps = pd.DataFrame()
timestamps["event_timestamp"] = pd.date_range("2020-12-01", "2020-12-07", freq='D')

In [21]:
entity_df = pd.merge(taxi_ids, timestamps, how='cross')
entity_df

,taxi_id,event_timestamp
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-01
1,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-02
2,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-03
3,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-04
4,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-05
...,...,...
35443,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-03
35444,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-04
35445,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-05
35446,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-06


In [22]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "trip_stats:total_miles_travelled",
        "trip_stats:total_trip_seconds",
        "trip_stats:total_earned",
        "trip_stats:trip_count",
        "on_demand_stats:avg_fare",
        "on_demand_stats:avg_trip_seconds",
        "on_demand_stats:avg_speed",
        "on_demand_stats:earned_per_hour",
    ]
)

Execute retrieval job with validation reference:

In [23]:
try:
    df = job.to_df(validation_reference=validation_reference)
except ValidationFailed as exc:
    print(exc.validation_report)

[
  {
    "success": false,
    "expectation_config": {
      "expectation_type": "expect_column_mean_to_be_between",
      "kwargs": {
        "column": "trip_count",
        "min_value": 10.387244591346153,
        "max_value": 12.695521167200855,
        "result_format": "COMPLETE"
      },
      "meta": {}
    },
    "result": {
      "observed_value": 6.692920555429092,
      "element_count": 4393,
      "missing_count": null,
      "missing_percent": null
    },
    "meta": {},
    "exception_info": {
      "raised_exception": false,
      "exception_message": null,
      "exception_traceback": null
    }
  },
  {
    "success": false,
    "expectation_config": {
      "expectation_type": "expect_column_mean_to_be_between",
      "kwargs": {
        "column": "earned_per_hour",
        "min_value": 52.32062497564023,
        "max_value": 63.9474305257825,
        "result_format": "COMPLETE"
      },
      "meta": {}
    },
    "result": {
      "observed_value": 68.99268345164135

Validation failed since several expectations didn't pass:
* Trip count (mean) decreased more than 10% (which is expected when comparing Dec 2020 vs June 2019)
* Average Fare increased - all quantiles are higher than expected
* Earn per hour (mean) increased more than 10% (most probably due to increased fare)